In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [2]:
#GPU device will have an id
torch.cuda.current_device()

0

In [3]:
#check how much memory is allocated on the GPU
#Pytorch will take more memory from the gpu
#This returns the current memory usage by the tensors
torch.cuda.memory_allocated()

0

In [4]:
#Current gpu memory managed
torch.cuda.memory_cached()

<ipython-input-4-87c1def07a81>:2: FutureWarning: `torch.cuda.memory_cached` has been renamed to `torch.cuda.memory_reserved`
  torch.cuda.memory_cached()


0

In [5]:
#For a cpu we would define torch tensors
a=torch.FloatTensor([1.0,2.0])

In [6]:
a

tensor([1., 2.])

In [7]:
a.device

device(type='cpu')

In [8]:
#Step 1 is to specify that we are using the gpu when we are creating the tensorts
a=torch.FloatTensor([1.0,2.0]).cuda()

In [9]:
a.device

device(type='cuda', index=0)

In [10]:
#Call .cuda for creating the tensors on the GPU
#Check how much memory is allocated
torch.cuda.memory_allocated()

512

In [11]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [12]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [13]:
torch.manual_seed(32)
model = Model()

In [14]:
# From the discussions here: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda

False

In [15]:
gpumodel = model.cuda()

In [16]:
next(gpumodel.parameters()).is_cuda

True

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the Iris dataset from the URL
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
df = pd.read_csv(url)

# Inspect the DataFrame to confirm column names
print(df.head())
print(df.columns)

# Use 'species' as the target column
X = df.drop('species', axis=1).values
y = df['species'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)


   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')
Training data shape: (120, 4)
Test data shape: (30, 4)


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# Load the Iris dataset from the URL
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
df = pd.read_csv(url)

# Features and target
X = df.drop('species', axis=1).values
y = df['species'].values

# Encode the species labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=33)

# Convert the features to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).cuda()
X_test_tensor = torch.FloatTensor(X_test).cuda()

# Convert the labels to PyTorch tensors (encoded as integers)
y_train_tensor = torch.LongTensor(y_train).cuda()
y_test_tensor = torch.LongTensor(y_test).cuda()

# Check shapes of tensors
print(f"X_train_tensor: {X_train_tensor.shape}")
print(f"y_train_tensor: {y_train_tensor.shape}")


X_train_tensor: torch.Size([120, 4])
y_train_tensor: torch.Size([120])


In [22]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [25]:
import torch
import time

# Ensure that X_train and y_train are PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).cuda()
y_train_tensor = torch.LongTensor(y_train).cuda()

# Training loop
epochs = 100
losses = []
start = time.time()

for i in range(epochs):
    i += 1
    y_pred = gpumodel.forward(X_train_tensor)  # Use the tensor instead of numpy array
    loss = criterion(y_pred, y_train_tensor)  # Use tensor for the target labels as well
    losses.append(loss)

    # a neat trick to save screen space:
    if i % 10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'TOTAL TRAINING TIME: {time.time()-start}')


epoch:  1  loss: 1.15074503
epoch: 11  loss: 0.93714464
epoch: 21  loss: 0.77962422
epoch: 31  loss: 0.60785323
epoch: 41  loss: 0.39894524
epoch: 51  loss: 0.25249201
epoch: 61  loss: 0.14927687
epoch: 71  loss: 0.10029557
epoch: 81  loss: 0.08100693
epoch: 91  loss: 0.07216036
TOTAL TRAINING TIME: 0.8279578685760498
